In [1]:
import torch
import torch.nn as nn
import cv2 
import numpy as np
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
import os
from tqdm import tqdm
import gc
import matplotlib.pyplot as plt
from src.calculate_metrics import F1
import segmentation_models_pytorch as smp

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='cuda', index=0)

In [3]:
# Thanks to https://github.com/Devyanshu/image-split-with-overlap/tree/master

class BatchesFromSingleImageMask:
    def __init__(self, imagefilename, batch_size, maskfilename, kernel_size=(1024, 1024), overlap=0.2, mode='train') -> None:
        self.imagefilename = imagefilename
        self.batch_size = batch_size
        self.kernel_size = kernel_size
        self.mode = mode

        ####################################
        # Augmentation here
        self.train_transform = A.Compose([
        
            A.OneOf(
                [
                    A.VerticalFlip(p=1),
                    A.HorizontalFlip(p=1),
                    A.RandomRotate90(p=1),
                    A.Transpose(p=1),
                ],
                p=0.5,
            ),
            ToTensorV2()
        ])
        #
        ####################################
        
        big_image = cv2.imread(imagefilename, cv2.IMREAD_COLOR)
        big_image = cv2.cvtColor(big_image, cv2.COLOR_BGR2RGB)
        
        if self.mode == 'train' or self.mode == 'val':
            big_mask = cv2.imread(maskfilename, cv2.IMREAD_GRAYSCALE)
            assert big_image.shape[0] == big_mask.shape[0] and big_image.shape[1] == big_mask.shape[1], "Image and mask shapes do not match"
            # Transform
            big_image, big_mask = self._transform(big_image, big_mask)
            
            self.masks = list(self._split_image(big_mask, kernel_size, overlap, is_mask=True))
            self.big_mask = big_mask.to(device)
        elif self.mode == 'test':
            big_image = torch.from_numpy(big_image).permute(2, 0, 1)
            big_image = big_image / 255.0
            
        self.orig_shape = big_image.shape[1:]
        self.big_image = big_image.to(device)
        self.images = list(self._split_image(big_image, kernel_size, overlap))
        self.generate_batches()

    def _transform(self, image, mask):
        self.transform = A.Compose([
                # A.RandomRotate90(p=0.5),
                ToTensorV2()
            ])
        
        augmented = self.transform(image=image, mask=mask)
        image = augmented['image'].float() / 255.0
        mask = augmented['mask']
        return image, mask
    
    def _split_image(self, big_image: torch.Tensor, desired_size: tuple=(512, 512), overlap=0.5, is_mask=False):
        """
            Generates the patches of smaller sizes from the big image.
            
        Args:
            big_image (tensor): An image with size bigger the desired size
            desired_size (tuple(int, int)): Desired size of the patches
            overlap (0 <= float < 1): the precentage of image to be overlapped
            is_mask (bool): Different behaviour for mask
        Returns:
            generator: Generator of ImageContainer objeects
        """
        if is_mask:
            big_image = big_image[None, :, :]
        img_c, img_h, img_w, = big_image.shape
        split_width, split_height = desired_size

        def start_points(size, split_size, overlap=0):
            points = [0]
            stride = int(split_size * (1-overlap))
            counter = 1
            while True:
                pt = stride * counter
                if pt + split_size >= size:
                    if split_size == size:
                        break
                    points.append(size - split_size)
                    break
                else:
                    points.append(pt)
                counter += 1
            return points

        X_points = start_points(img_w, split_width, overlap)
        Y_points = start_points(img_h, split_height, overlap)

        for i in Y_points:
            for j in X_points:
                yield (i, j)
    
    def generate_batches(self):
        self.batches = []
        if self.mode == 'train':
            # Train mode
            for i in range(0, len(self.images), self.batch_size)[:-1]:
                image_stack, mask_stack = [], []
                for j in range(self.batch_size):
                    image_slice = self._get_image_slice(self.images[i + j]).permute(1, 2, 0).cpu().numpy()
                    mask_slice = self._get_mask_slice(self.images[i + j]).cpu().numpy()

                    augmented = self.train_transform(image=image_slice, mask=mask_slice)
                    image = augmented['image']
                    mask = augmented['mask']

                    image_stack.append(image)
                    mask_stack.append(mask)
                self.batches.append([
                    torch.stack(image_stack),
                    torch.stack(mask_stack)
                ])
            image_stack, mask_stack = [], []
            for j in range(self.batch_size):
                image_slice = self._get_image_slice(self.images[len(self.images) - self.batch_size + j]).permute(1, 2, 0).cpu().numpy()
                mask_slice = self._get_mask_slice(self.images[len(self.masks) - self.batch_size + j]).cpu().numpy()
                
                augmented = self.train_transform(image=image_slice, mask=mask_slice)
                image = augmented['image']
                mask = augmented['mask']

                image_stack.append(image)
                mask_stack.append(mask)
            self.batches.append([
                torch.stack(image_stack),
                torch.stack(mask_stack)
            ])
        elif self.mode == 'test':
            # Test mode
            for i in range(0, len(self.images), self.batch_size)[:-1]:
                self.batches.append(
                    torch.stack(tuple(self._get_image_slice(self.images[i + j]) for j in range(self.batch_size)))
                )
            self.batches.append(
                torch.stack(tuple(self._get_image_slice(self.images[len(self.images) - self.batch_size + j]) for j in range(self.batch_size)))
            )
        else:
            # Val mode
            for i in range(0, len(self.images), self.batch_size)[:-1]:
                self.batches.append(
                    (torch.stack(tuple(self._get_image_slice(self.images[i + j]) for j in range(self.batch_size))),
                    torch.stack(tuple(self._get_mask_slice(self.masks[i + j]) for j in range(self.batch_size))))
                )
            self.batches.append(
                (torch.stack(tuple(self._get_image_slice(self.images[len(self.images) - self.batch_size + j]) for j in range(self.batch_size))),
                torch.stack(tuple(self._get_mask_slice(self.masks[len(self.masks) - self.batch_size + j]) for j in range(self.batch_size))))
            )
        return self.batches

    def _get_image_slice(self, pos):
        posy, posx = pos
        dh, dw = self.kernel_size
        return self.big_image[:, posy:posy+dh, posx:posx+dw]

    def _get_mask_slice(self, pos):
        posy, posx = pos
        dh, dw = self.kernel_size
        return self.big_mask[posy:posy+dh, posx:posx+dw]
    
    def __getitem__(self, idx):
        return self.batches[idx]
    
    def __len__(self):
        return len(self.batches)

    def merge_masks(self, masks):
        answer = torch.zeros(size=(2, *self.orig_shape)).to(device)
        counter = torch.zeros(size=(2, *self.orig_shape)).to(device)
        k = 0
        for batch in masks[:-1]:
            for mask in batch:
                posy, posx = self.images[k]
                dh, dw = self.kernel_size
                answer[:, posy: posy + dh, posx: posx + dw] += mask
                counter[:, posy: posy + dh, posx: posx + dw] += 1
                k += 1
        k = len(self.images) - self.batch_size
        for mask in masks[-1]:
            posy, posx = self.images[k]
            dh, dw = self.kernel_size
            answer[:, posy: posy + dh, posx: posx + dw] += mask
            counter[:, posy: posy + dh, posx: posx + dw] += 1
            k += 1
        answer = answer / counter
        answer = torch.argmax(answer, dim = 0, keepdims=True)
        # Convert to HWC
        answer = answer.permute(1, 2, 0)
        return answer

Train / Validation

In [4]:
import random

seed_value = 42
random.seed(seed_value)

filenames = [(f"train/images/train_image_{i:03d}.png", f"train/masks/train_mask_{i:03d}.png") for i in range(21)]
random.shuffle(filenames)

train_size = int(0.8 * len(filenames))

train_filenames = filenames[:train_size]
val_filenames = filenames[train_size:]


Model

In [5]:
from src.ensemble import EnsembleModel

model = EnsembleModel([
    smp.UnetPlusPlus('resnet50', encoder_weights='imagenet', activation=None, classes = 2, encoder_depth= 5, decoder_channels=[256, 128, 64, 32, 16]),
    smp.UnetPlusPlus('efficientnet-b7', encoder_weights='imagenet', activation=None, classes = 2, encoder_depth= 5, decoder_channels=[256, 128, 64, 32, 16])
])

model = model.to(device)

Train

In [6]:
from src.DiceLoss import DiceLoss

optimizer = torch.optim.Adam(
    model.parameters(),
)

epochs = 30

optimizer = torch.optim.Adam(model.parameters())

criterion1 = nn.CrossEntropyLoss(reduction="mean")
criterion2 = DiceLoss()

In [7]:
history = {
    'train_loss': [],
    'val_loss': [],
    'F1 score': [],
}

def train_model(model, train_filenames, val_filenames, device, optimizer, epochs=1, batch_size=2):
    best_score = 0
    for epoch in range(1, epochs + 1):
        model.train()
        epoch_loss = 0
        loss_len = 0
        with tqdm(
            desc=f"Epoch {epoch}/{epochs}", unit="img"
        ) as pbar:
            for im, mk in train_filenames:
                train_loader = BatchesFromSingleImageMask(im, batch_size, mk, mode='train')
                for i, batch in enumerate(train_loader):
                    images, true_masks = batch
        
                    images, true_masks = images.to(device), true_masks.to(device)
                    true_masks = true_masks.squeeze(dim=1)
                    
                    masks_pred = model(images)
                    loss = criterion1(masks_pred, true_masks.long()) + criterion2(masks_pred, true_masks)
                    epoch_loss += loss.item()
                    loss_len += 1
                    
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    
                    pbar.update(images.shape[0])
                    pbar.set_description(f"Epoch: {epoch}, train_loss: {epoch_loss / (loss_len)}")

                del train_loader
                gc.collect()
            history['train_loss'].append(epoch_loss)
            
                
        model.eval()
        f1_metrics = np.array([])
        epoch_loss = 0
        loss_len = 0
        
        with tqdm(desc=f"Validation", unit="img") as pbar:
            with torch.no_grad():
                for im, mk in val_filenames:
                    val_loader = BatchesFromSingleImageMask(im, batch_size, mk, mode='val')
        
                    predictions_batches = []
                    
                    
                    for i, batch in enumerate(val_loader):
                        images, true_masks = batch
        
                        images, true_masks = images.to(device), true_masks.to(device)
                        true_masks = true_masks.squeeze(dim=1)
                        
                        masks_pred = model(images)
                        
                        loss = criterion1(masks_pred, true_masks.long()) + criterion2(masks_pred, true_masks)
                        epoch_loss += loss.item()
                        loss_len += 1
                        
                        predictions_batches.append(masks_pred)
                        
                        pbar.update(images.shape[0])
                        pbar.set_description(f"val_loss: {epoch_loss / (loss_len)}")
                    
                    pred_full = val_loader.merge_masks(predictions_batches).permute(2, 0, 1)
                    true_full = val_loader.big_mask[None,:,:]
                    
                    f1_metrics = np.append(f1_metrics, F1(true_full, pred_full).cpu().numpy())
                    
        F1_score = f1_metrics.mean()
        print(f'F1 score: {F1_score}')
        history['val_loss'].append(epoch_loss)
        history['F1 score'].append(F1_score)
        
        if F1_score > best_score:
            best_score = F1_score
            torch.save(model.state_dict(), "best_so_far_aug.pt")
    return history

In [8]:
history = train_model(model, train_filenames, val_filenames, device, optimizer, epochs=epochs)

Epoch: 1, train_loss: 0.6882905835774963: : 990img [06:47,  2.43img/s]
val_loss: 0.606541684709373: : 320img [00:38,  8.37img/s] 


F1 score: 0.5679290592670441


Epoch: 2, train_loss: 0.5587540461296084: : 990img [07:04,  2.33img/s]
val_loss: 0.5304873357858014: : 320img [00:38,  8.34img/s] 


F1 score: 0.628374433517456


Epoch: 3, train_loss: 0.5058067934013752: : 990img [06:40,  2.47img/s] 
val_loss: 0.5643281044045579: : 320img [00:37,  8.65img/s] 


...

F1 score: 0.6142610728740692


Epoch: 29, train_loss: 0.3085187513404467: : 990img [07:49,  2.11img/s] 
val_loss: 0.49356872092030246: : 320img [00:47,  6.77img/s]


F1 score: 0.6182458221912384


Epoch: 30, train_loss: 0.29117528009298965: : 990img [08:03,  2.05img/s]
val_loss: 0.5095457875766443: : 320img [00:44,  7.23img/s] 

F1 score: 0.6011254638433456


In [ ]:
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.plot(history['train_loss'], label='Train Loss', marker='o')
plt.plot(history['val_loss'], label='Validation Loss', marker='o')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plotting F1 score
plt.subplot(1, 2, 2)
plt.plot(history['F1 score'], label='F1 Score', marker='o', color='green')
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.title('F1 Score over Epochs')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
model = EnsembleModel([
    smp.UnetPlusPlus('resnet50', encoder_weights='imagenet', classes = 2, activation=None, encoder_depth= 5, decoder_channels=[256, 128, 64, 32, 16]),
    smp.UnetPlusPlus('efficientnet-b7', encoder_weights='imagenet', classes = 2, activation=None, encoder_depth= 5, decoder_channels=[256, 128, 64, 32, 16]),
])
model = model.to(device)

In [13]:
model.load_state_dict(torch.load('best_unetplusplus_ensemble.pt'))

<All keys matched successfully>

## **Post-Processing**

In [52]:
def delete_small_buildings(image, min_size):
    # Find connected components in the image
    _, labels, stats, _ = cv2.connectedComponentsWithStats(image, connectivity=8)

    # Iterate through connected components and remove small ones
    for i in range(1, stats.shape[0]):  # Skip the background (label 0)
        if stats[i, cv2.CC_STAT_AREA] < min_size:
            image[labels == i] = 0

    return torch.tensor(image).unsqueeze(0)

In [53]:
def validation(min_building_size, batch_size=3):
    model.eval()
    f1_metrics = np.array([])
    
    with tqdm(desc=f"Validation", unit="img") as pbar:
        with torch.no_grad():
            for im, mk in val_filenames:
                val_loader = BatchesFromSingleImageMask(im, batch_size, mk, mode='val')
    
                predictions_batches = []
                
                
                for i, batch in enumerate(val_loader):
                    images, true_masks = batch
    
                    images, true_masks = images.to(device), true_masks.to(device)
                    true_masks = true_masks.squeeze(dim=1)
                    
                    masks_pred = model(images)
                    
                    predictions_batches.append(masks_pred)
                    
                    pbar.update(images.shape[0])
                
                pred_full = val_loader.merge_masks(predictions_batches).permute(2, 0, 1)
                true_full = val_loader.big_mask[None,:,:]
                
                pred_full = delete_small_buildings(pred_full.squeeze(0).cpu().numpy().astype(np.uint8), min_building_size)
                pred_full = pred_full.to(device)
                f1_metrics = np.append(f1_metrics, F1(true_full, pred_full).cpu().numpy())
                
    F1_score = f1_metrics.mean()
    print(f'F1 score: {F1_score}')
    return F1_score

In [54]:
min_building_size = np.linspace(0, 300, num=31, dtype=np.int32)
best_f1_score, best_min_size = 0, 0

for min_size in min_building_size:
    print(f'{min_size=}')
    score = validation(min_size)
    if score > best_f1_score:
        best_f1_score = score
        best_min_size = min_size
print(f'{best_f1_score=}, {best_min_size=}')

min_size=0


Validation: 387img [00:45,  8.49img/s]


F1 score: 0.7006216526031495
min_size=10


Validation: 387img [00:51,  7.46img/s]


F1 score: 0.7006275296211243
min_size=20


...

F1 score: 0.7013757228851318
min_size=290


Validation: 387img [02:32,  2.54img/s]


F1 score: 0.7012992620468139
min_size=300


Validation: 387img [02:21,  2.74img/s]

F1 score: 0.7012162923812866
best_f1_score=0.7015440702438355, best_min_size=240


## **Inference**

In [ ]:
model.eval()
batch_size = 1
test_filenames = [f"test/images/test_image_{i:03d}.png" for i in range(21)]

with tqdm(desc=f"Inference", unit="img") as pbar:
    with torch.no_grad():
        for j, im in enumerate(test_filenames):
            test_loader = BatchesFromSingleImageMask(im, batch_size, None, test_mode=True)
            
            predictions_batches = []
            
            for i, images in enumerate(test_loader.generate_batches()):
                images = images.to(device)
                masks_pred = model(images)

                pred = torch.argmax(masks_pred, dim=1)
            
                predictions_batches.append(masks_pred)
                
                pbar.update(images.shape[0])
            pred_full = test_loader.merge_masks(predictions_batches).permute(2, 0, 1)
            np_pred_full = pred_full.squeeze(0).cpu().numpy()
            plt.imsave(f'test/masks/test_image_{j:03d}.png', np_pred_full)
            

In [ ]:
def plot_result(original_image, prediction_mask, true_mask):
    original_image = cv2.imread(original_image cv2.IMREAD_COLOR)
    original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
    
    prediction_mask = prediction_mask.cpu().numpy()
    true_mask = true_mask.cpu().numpy()
    
    fig = plt.figure()
    plt.imshow(original_image)
    plt.imshow(true_mask, alpha=0.6)
    plt.title('Original mask')
    plt.show()
    # fig.savefig('figure_real.png', dpi=500, bbox_inches='tight')

    fig = plt.figure()
    plt.imshow(original_image)
    plt.imshow(prediction_maskk, alpha=0.6)
    plt.title('Prediction mask')
    plt.show()
    # fig.savefig('figure_predicted.png', dpi=500, bbox_inches='tight')